In [ ]:
import cracknuts as cn

In [ ]:
def bytes_bin(b: bytes, sep: str = ' ') -> str:
    return sep.join(f'{byte:08b}' for byte in b)

In [ ]:
s1 = cn.new_cracker('192.168.0.29')
s1.connect(force_update_bin=True, force_write_default_config=True)

In [ ]:
s1.get_firmware_version()

In [ ]:
# 复位 PL IO
s1.register_write(base_address=0x43c10000, offset=0x300, data=0x01)

In [ ]:
# 使能 PL IO
s1.register_write(base_address=0x43c10000, offset=0x304, data=0b0000_0000_0000_0000_0000_0000_0000_0000)
s1.register_write(base_address=0x43c10000, offset=0x304, data=0b0000_0000_0000_0000_0000_0000_0001_1111)

In [ ]:
# 输入输出控制
s1.register_write(base_address=0x43c10000, offset=0x308, data=0b0000_0000_0000_0000_0000_0000_0000_0000) # 输出
# s1.register_write(base_address=0x43c10000, offset=0x308, data=0b0000_0000_0000_0000_0000_0000_0000_1111) # 输入

In [ ]:
s1.register_write(base_address=0x43c10000, offset=0x318, data=0b0000_0000_0000_0000_0000_0000_0000_0101) # 波形

In [ ]:
# 设置波形长度
s1.register_write(base_address=0x43c10000, offset=0x32c, data=200_000_000) 
s1.register_write(base_address=0x43c10000, offset=0x330, data=200_000_000) 
s1.register_write(base_address=0x43c10000, offset=0x334, data=200_000_000) 
s1.register_write(base_address=0x43c10000, offset=0x338, data=200_000_000) 

In [ ]:
# 设置波形开始
s1.register_write(base_address=0x43c10000, offset=0x30c, data=0b0000_0000_0000_0000_0000_0000_0001_1111)

In [ ]:
# 设置输入模式
s1.register_write(base_address=0x43c10000, offset=0x308, data=0b0000_0000_0000_0000_0000_0000_0000_1111) # 输入

In [ ]:
# 设置输入使能
s1.register_write(base_address=0x43c10000, offset=0x310, data=0b0000_0000_0000_0000_0000_0000_0000_1111) # mask使能
s1.register_write(base_address=0x43c10000, offset=0x314, data=0b0000_0000_0000_0000_0000_0000_0000_1111) # 或 模式

In [ ]:
import random
import time
from cracknuts.cracker import serial


cmd_set_aes_enc_key = "01 00 00 00 00 00 00 10"
cmd_aes_enc = "01 02 00 00 00 00 00 10"

aes_key = "11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff"
aes_data_len = 16

sample_length = 20000

def init(c):
    s1.nut_voltage_enable()
    s1.nut_voltage(3.3)
    s1.nut_clock_enable()
    s1.nut_clock_freq('8M')
    s1.uart_io_enable()
    s1.osc_sample_clock('48m')
    s1.osc_sample_length(sample_length)
    s1.osc_trigger_source('N')
    s1.osc_analog_gain('B', 10)
    s1.osc_trigger_level(0)
    s1.osc_trigger_mode('E')
    s1.osc_trigger_edge('U')
    s1.uart_config(baudrate=serial.Baudrate.BAUDRATE_115200, bytesize=serial.Bytesize.EIGHTBITS, parity=serial.Parity.PARITY_NONE, stopbits=serial.Stopbits.STOPBITS_ONE)

    time.sleep(2)
    cmd = cmd_set_aes_enc_key + aes_key
    status, ret = s1.uart_transmit_receive(cmd, timeout=1000, rx_count=6)
    time.sleep(2)

def do(cracer, count):
    plaintext_data = random.randbytes(aes_data_len)
    tx_data = bytes.fromhex(cmd_aes_enc.replace(' ', '')) + plaintext_data
    status, ret = s1.uart_transmit_receive(tx_data, rx_count= 6 + aes_data_len, is_trigger=True)
    
    return {
        "plaintext": plaintext_data,
        "ciphertext": ret[-aes_data_len:],
        "key": bytes.fromhex(aes_key)
    }


def finish(c):
    ...
    # print('optional behavior')


acq = cn.new_acquisition(s1, init=init, do=do, finish=finish)
cn.panel(acq)